In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
from scipy.optimize import minimize

from datafold.appfold import EDMDControl
from datafold.dynfold.dmd import ControlledAffineDynamicalSystem, gDMDAffine
from datafold.dynfold.transform import TSCPolynomialFeatures, TSCRadialBasis
from datafold.pcfold import InitialCondition, InverseQuadraticKernel, TSCDataFrame
from datafold.utils._systems import InvertedPendulum
from datafold.utils.general import if1dim_colvec, if1dim_rowvec

## Predictor

### Random system of the form `xdot = A @ x + B @ u @ x`

In [ ]:
state_size = 4
input_size = 2
n_timesteps = 50
n_ic = 1

gen = np.random.default_rng(22)

A = gen.uniform(-0.5, 0.5, size=(state_size, state_size))
np.fill_diagonal(A, gen.uniform(-1.0, -0.5, size=state_size))
x0 = gen.uniform(-1.0, 1.0, size=(state_size, n_ic))
x0 = np.hstack([x0, x0, x0, x0])
Bi = np.stack(
    [gen.uniform(-0.5, 0.5, size=(state_size, state_size)) for i in range(input_size)],
    2,
)
u = np.concatenate(
    [
        np.ones((n_ic, n_timesteps, input_size)),
        -np.ones((n_ic, n_timesteps, input_size)),
        np.stack([np.ones((n_ic, n_timesteps)), -np.ones((n_ic, n_timesteps))], 2),
        np.stack([-np.ones((n_ic, n_timesteps)), np.ones((n_ic, n_timesteps))], 2),
    ],
    0,
)
t = np.linspace(0, n_timesteps - 1, n_timesteps) * 0.1
names = ["x" + str(i + 1) for i in range(state_size)]

tsc_df = (
    ControlledAffineDynamicalSystem()
    .setup_matrix_system(A, Bi)
    .evolve_system(x0, u, time_values=t, time_delta=0.1, feature_names_out=names)
)
# print(tsc_df)
# print(A)
# print(Bi)
ureshaped = u.reshape((-1, input_size))
for i in range(input_size):
    tsc_df["u" + str(i + 1)] = ureshaped[:, i]
tsc_df.plot()

In [ ]:
dmd = gDMDAffine().fit(tsc_df, split_by="name", control=["u1", "u2"])

In [ ]:
tsc_pred = dmd.fit_predict(tsc_df)
tsc_pred.plot()

In [ ]:
(tsc_df - tsc_pred).plot()

### Duffing

In [ ]:
def duffing(t, x, alpha=-1, beta=1, delta=0.6, u=lambda t: 1):
    xdot = np.array([x[1], 0])
    xdot[1] = -delta * x[1] - alpha * x[0] - beta * x[0] ** 3 + u(t)
    return xdot


def simulate(x0, u=lambda t: 1, N=201, t0=0, tf=2):
    sol = solve_ivp(
        lambda t, x: duffing(t, x, u=u), (t0, tf), x0, t_eval=np.linspace(t0, tf, N)
    )
    if not sol.success:
        raise RuntimeError("Couldn't not evolve the system.")
    df = pd.DataFrame(data=sol.y.T, index=sol.t, columns=["x", "xdot"])
    df["u"] = u(sol.t)
    return df


x0 = np.array([0.1, -0.9])
x = simulate(x0, lambda t: -1)
plt.plot(x["x"])
x = simulate(x0)
plt.plot(x["x"])
x = simulate(x0, lambda t: np.sin(np.pi * t))
plt.plot(x["x"], "purple")
x = simulate(x0, lambda t: 0)
plt.plot(x["x"], "gold")

In [ ]:
n_repeats = 100
rng = np.random.default_rng(0)
dflist = []
for i in range(n_repeats):
    random_x0 = rng.uniform(-3, 3, 2)
    dflist.append(simulate(random_x0, N=21))
    dflist.append(simulate(random_x0, lambda t: -1, N=21))
duffing_tsc = TSCDataFrame.from_frame_list(dflist)
duffing_tsc.plot()

In [ ]:
edmd_duffing = EDMDControl(
    dict_steps=[
        ("poly", TSCPolynomialFeatures(5, include_bias=True)),
    ],
    dmd_model=gDMDAffine(rcond=1e-6),
)
edmd_duffing.fit(duffing_tsc, split_by="name", control=["u"]);

In [ ]:
plt.figure(figsize=(8, 3))
plt.subplot(121)
plt.imshow(edmd_duffing.sys_matrix)
plt.colorbar()
plt.title("System matrix")
print("Sys eigenvalues: ", np.linalg.eigvals(edmd_duffing.sys_matrix))
plt.subplot(122)
plt.imshow(edmd_duffing.control_matrix[:, :, 0])
plt.colorbar()
plt.title("Control matrix")
print("Control eigenvalues: ", np.linalg.eigvals(edmd_duffing.control_matrix[:, :, 0]))

In [ ]:
n_samples = 50
t = np.linspace(0, 2, n_samples)
pred0 = edmd_duffing.predict(
    InitialCondition.from_array(x0, columns=["x", "xdot"]),
    time_values=t,
    control_input=-np.ones(n_samples),
)
plt.plot(t, pred0["x"].values, "tab:blue", label=r"$u=-1$", linestyle="--")
pred0 = edmd_duffing.predict(
    InitialCondition.from_array(x0, columns=["x", "xdot"]),
    time_values=t,
    control_input=np.ones(n_samples),
)
plt.plot(t, pred0["x"].values, "tab:red", label=r"$u=1$", linestyle="--")
pred0 = edmd_duffing.predict(
    InitialCondition.from_array(x0, columns=["x", "xdot"]),
    time_values=t,
    control_input=np.sin(np.pi * t),
)
plt.plot(t, pred0["x"].values, "purple", label=r"$u=sin(\pi t)$", linestyle="--")
pred0 = edmd_duffing.predict(
    InitialCondition.from_array(x0, columns=["x", "xdot"]),
    time_values=t,
    control_input=0 * np.ones(n_samples),
)
plt.plot(t, pred0["x"].values, "gold", label=r"$u=0$", linestyle="--")
x = simulate(x0, lambda t: -1)
plt.plot(x["x"], "tab:blue")
x = simulate(x0)
plt.plot(x["x"], "tab:red")
x = simulate(x0, lambda t: np.sin(np.pi * t))
plt.plot(x["x"], "purple")
x = simulate(x0, lambda t: 0)
plt.plot(x["x"], "gold")
plt.legend();

### Inverted Pendulum

In [ ]:
state_cols = ["x", "xdot", "theta", "thetadot"]
control_cols = ["u"]

# Data generation parameters
sim_time_step = 0.01  # s
sim_num_steps = 1000  # -
training_size = 20  # -
ic = InitialCondition.from_array(np.array([0, 0, np.pi, 0]), columns=state_cols)

invertedPendulum = InvertedPendulum(initial_condition=ic.values)

Xlist, Ulist = [], []
np.random.seed(42)
for i in range(training_size):
    control_amplitude = 0.1 + 0.9 * np.random.random()
    control_frequency = np.pi + 2 * np.pi * np.random.random()
    control_phase = 2 * np.pi * np.random.random()
    control_func = lambda t, y: control_amplitude * np.sin(
        control_frequency * t + control_phase
    )
    invertedPendulum.reset()
    traj = invertedPendulum.predict(
        time_step=sim_time_step,
        num_steps=sim_num_steps,
        control_func=control_func,
    )
    assert (
        invertedPendulum.sol.success
    ), f"Divergent solution for amplitude={control_amplitude}, frequency={control_frequency}"
    t = invertedPendulum.sol.t
    dfx = pd.DataFrame(data=traj.T, index=t, columns=state_cols)
    dfx[control_cols] = 0.0
    Xlist.append(dfx)
    control_input = control_func(t, traj)
    dfu = pd.DataFrame(data=control_input, index=t, columns=control_cols)
    for col in state_cols:
        dfu[col] = 0.0
    dfu = dfu[state_cols + control_cols]
    Ulist.append(dfu)

X_tsc = TSCDataFrame.from_frame_list(Xlist)[state_cols]
X_tsc[control_cols] = TSCDataFrame.from_frame_list(Ulist)[control_cols]

In [ ]:
num_rbfs = 20
eps = 1

rbf = TSCRadialBasis(
    kernel=InverseQuadraticKernel(epsilon=eps), center_type="fit_params"
)
center_ids = sorted(
    np.random.choice(
        range(0, sim_num_steps * training_size), size=num_rbfs, replace=False
    )
)
centers = X_tsc.iloc[center_ids].values

In [ ]:
edmdrbf = EDMDControl(
    dict_steps=[
        ("rbf", rbf),
    ],
    include_id_state=True,
    dmd_model=gDMDAffine(rcond=1e-6),
)

edmdrbf.fit(
    X_tsc,
    split_by="name",
    state=state_cols,
    control=control_cols,
    rbf__centers=centers[:, :-1],
)
rbfprediction = edmdrbf.predict(
    ic, control_input=np.atleast_2d(control_input).T, time_values=t
)
plt.figure(figsize=(16, 3))
plt.subplot(121)
plt.plot(rbfprediction["x"].values[:100], label="prediction")
plt.plot(dfx["x"].values[:100], label="actual")
plt.legend()
plt.title(r"EDMD(100 random rbf) prediction - cart position $x$")
plt.subplot(122)
plt.plot(rbfprediction["theta"].values[:100], label="prediction")
plt.plot(dfx["theta"].values[:100], label="actual")
plt.legend()
plt.title(r"EDMD(100 random rbf) prediction - pendulum angle $\theta$");

In [ ]:
edmdrbf.predict(
    ic, control_input=np.atleast_2d(control_input[:10]).T, time_values=t[:10]
)

## KMPC

### Implementation

In [ ]:
class KMPC_Peitz(object):
    def __init__(
        self,
        predictor: EDMDControl,
        horizon: int,
        input_bounds: np.array,
        cost_state: np.array,
        cost_input: np.array,
    ):
        self.predictor = predictor

        self.horizon = horizon

        self.L = predictor.sys_matrix
        self.B = predictor.control_matrix
        self.lifted_state_size, _, self.input_size = self.B.shape
        self.state_size = len(predictor.state_columns)

        self.input_bounds = np.repeat(input_bounds.T, self.horizon + 1, axis=1).T
        self.cost_state = cost_state
        self.cost_input = cost_input
        self._cached_input = None
        self._cached_state = None
        self._cached_prediction = None

    def _predict(self, x0, u, t):
        # -> shape (self.lifted_state_size, self.horizon+1)
        if (self._cached_input != u).any() or (self._cached_state != x0).any:
            tsc = self.predictor.predict(
                InitialCondition.from_array(x0.T, self.predictor.state_columns),
                control_input=u.T[np.newaxis],
                time_values=t,
                lifted_state=True,
                check_inputs=False,
            )
            self._cached_prediction = tsc.values.T
            self._cached_input = u
            self._cached_state = x0
        return self._cached_prediction

    def cost(self, u, x0, xref, t):
        """_summary_

        Parameters
        ----------
        u : np.array
            shape = (n*m,)
            [u1(t0) u1(t1) ... u1(tn) u2(t1) ... um(tn)]
            with `n = self.horizon+1`; `m = self.input_size`
        x0 : np.array
            shape = `(self.state_size, 1)`
        xref : np.array
            shape = `(self.state_size, 1)`

        Returns
        ---------
        float
        """
        u = u.reshape(self.input_size, self.horizon + 1)
        x = self._predict(x0, u, t)[: self.state_size, :]
        Lhat = self.cost_state * np.linalg.norm(
            x - xref, axis=0
        ) + self.cost_input * np.linalg.norm(u, axis=0)
        J = np.sum(Lhat)
        # print('J', J)
        # print('u', u)
        # print('x', x)
        return J

    def jacobian(self, u, x0, xref, t):
        """_summary_

        Parameters
        ----------
        u : np.array
            shape = (n*m,)
            [u1(t0) u1(t1) ... u1(tn) u2(t1) ... um(tn)]
            with `n = self.horizon+1`; `m = self.input_size`
        x0 : np.array
            shape = `(self.state_size, 1)`
        xref : np.array
            shape = `(self.state_size, 1)`

        Returns
        ---------
        array_like, shape (n*m,)
        [dJ/du1(t0) dJ/du1(t1) ... dJ/du1(tn) dJ/du2(t1) ... dJ/dum(tn)]
        """
        u = u.reshape(self.input_size, self.horizon + 1)
        x = self._predict(x0, u, t)  # shape = (lifted_state_size, horizon+1)
        interp_gamma = interp1d(t, self._dcost_dx(x, xref), axis=1)
        interp_u = interp1d(t, u, axis=1)

        lambda_adjoint = self._compute_lambda_adjoint(
            interp_u, interp_gamma, t
        )  # shape = (lifted_state_size,horizon+1)
        rho = self._dcost_du(u)  # shape = (input_size, horizon+1)

        # self.B.shape = (lifted_state_size, lifted_state_size, input_size)
        # (x.T @ self.B.T).shape = (input_size, horizon+1, lifted_state_size)
        # einsum(...).shape = rho.shape = (input_size, horizon+1)
        jac = np.einsum("ijk,kj->ij", x.T @ self.B.T, lambda_adjoint) + rho
        # print('jac', jac)
        # print(lambda_adjoint)
        return jac.ravel()

    def _lambda_ajoint_ODE(self, t, y, interp_u, interp_gamma):
        return (
            -interp_gamma(t)
            - self.L.T @ y
            - np.einsum("ijk,i->jk", self.B.T, interp_u(t)) @ y
        )

    def _compute_lambda_adjoint(self, interp_u, interp_gamma, t):
        sol = solve_ivp(
            self._lambda_ajoint_ODE,
            y0=np.zeros(self.lifted_state_size),
            t_span=[t[-1], t[0]],
            t_eval=t[::-1],
            args=(interp_u, interp_gamma),
        )
        if not sol.success:
            raise RuntimeError(
                "Could not integrate the adjoint dynamics. Solver says '{sol.message}'"
            )
        return np.fliplr(sol.y)

    def _dcost_dx(self, x, xref):
        # gamma(t0:te)
        gamma = np.zeros((self.lifted_state_size, self.horizon + 1))
        gamma[: self.state_size, :] = (
            self.cost_state * 2 * (x[: self.state_size, :] - xref)
        )
        return gamma

    def _dcost_du(self, u):
        # rho(t0:te)
        return self.cost_input * 2 * (u)

    def generate_control_signal(self, x0, xref, t, **minimization_kwargs):
        # x0.shape = (nc,1)
        # xref.shape = (nc,n)
        # t.shape = (n,)
        if t.shape != (self.horizon + 1,):
            raise ValueError(f"t is of shape {t.shape}, should be ({self.horizon+1},)")
        u_init = np.zeros(
            (self.input_size, self.horizon + 1)
        ).ravel()  # [u1(t1) u1(t2) ... u1(tn) u2(t1) ... um(tn)]
        res = minimize(
            fun=self.cost,
            x0=u_init,
            args=(if1dim_colvec(x0), if1dim_colvec(xref), t),
            method="L-BFGS-B",
            jac=self.jacobian,
            bounds=self.input_bounds,
            **minimization_kwargs,
        )

        if not res.success:
            raise RuntimeError(
                f"Could not find a minimum solution. Solver says '{res.message}'"
            )

        return res.x.reshape(self.input_size, self.horizon + 1)

### Duffing

In [ ]:
horizon_size = 5
horizon_time = 1
kmpc_duffing = KMPC_Peitz(
    edmd_duffing,
    horizon=horizon_size,
    input_bounds=np.array([[-1, 1]]),
    cost_state=1,
    cost_input=0,
)
x0 = np.array([0.15, -0.9])
t = np.linspace(0, horizon_time, horizon_size + 1)
xref = np.zeros((2, horizon_size + 1))
# xref[0,100:200] = 1.0
# xref[0,200:300] = -1.0
%prun -s "cumulative" u = kmpc_duffing.generate_control_signal(x0,xref,t,tol=1e-3)

#### Long horizon

In [ ]:
horizon_size = 20
horizon_time = 5
kmpc_duffing = KMPC_Peitz(
    edmd_duffing,
    horizon=horizon_size,
    input_bounds=np.array([[-1, 1]]),
    cost_state=1,
    cost_input=0,
)
x0 = np.array([0.15, -0.9])
t = np.linspace(0, horizon_time, horizon_size + 1)
xref = np.zeros((2, horizon_size + 1))
# xref[0,100:200] = 1.0
# xref[0,200:300] = -1.0
u = kmpc_duffing.generate_control_signal(x0, xref, t, tol=1e-3)

In [ ]:
edmd_duffing.predict(x0, time_values=t, control_input=u.T).plot()
plt.plot(t, u.T, color="g")

In [ ]:
u_interp = interp1d(t, u)
x = simulate(x0, lambda t: u_interp(t).T, tf=5)
x.plot()

#### Sliding window

In [ ]:
horizon_size = 5
dt = 0.4
horizon_time = horizon_size * dt
total_time = 20
timesteps = int(total_time / dt)
t = np.arange(timesteps + 1) * dt
ulist = []
xlist = []
xref = np.zeros((2, timesteps + 1))
xref[0, 25:50] = 1.0
# xref[0,200:300] = -1.0
x0 = np.array([0.1, -0.9])
kmpc_duffing = KMPC_Peitz(
    edmd_duffing,
    horizon=horizon_size,
    input_bounds=1.1 * np.array([[-1, 1]]),
    cost_state=1,
    cost_input=0,
)
xx0 = x0
for i in range(int(total_time / horizon_time)):
    xxref = xref[:, i * horizon_size : (i + 1) * horizon_size + 1]
    tt = t[i * horizon_size : (i + 1) * horizon_size + 1]
    u = kmpc_duffing.generate_control_signal(
        xx0, xxref, tt, tol=1e-3, options={"ftol": 1e-3}
    )
    u_interp = interp1d(tt, u)
    xx = simulate(xx0, lambda t: u_interp(t).T, N=horizon_size + 1, t0=tt[0], tf=tt[-1])
    xx0 = xx[["x", "xdot"]].values[-1]
    print(f"{i}/{int(total_time/horizon_time)}")
    xlist.append(xx.iloc[:-1])

In [ ]:
TSCDataFrame.from_frame_list(xlist).plot()

## Appendix 
### Einsum test

In [ ]:
np.einsum(
    "ij,ik->ijk",
    np.array([[1, -1], [2, -2], [3, -3]]),
    np.array([[0.1, 0.2, 0.3, 0.4], [1.1, 1.2, 1.3, 1.4], [10.1, 12.1, 13.1, 14.1]]),
).reshape(3, 2 * 4)

In [ ]:
np.einsum(
    "ij,ik->ijk",
    np.array([[0.1, 0.2, 0.3, 0.4], [1.1, 1.2, 1.3, 1.4], [10.1, 12.1, 13.1, 14.1]]),
    np.array([[1, -1], [2, -2], [3, -3]]),
).reshape(3, 2 * 4)

### IVP backwards test

In [ ]:
sol = solve_ivp(
    lambda t, y: np.cos(t),
    y0=np.array([0]),
    t_span=[5, 0.0],
    t_eval=np.linspace(5, 0.0),
)
plt.plot(sol.t, sol.y.T)
plt.plot(np.linspace(0, 5), sol.y.T)
np.fliplr(sol.y)

### Einsum test 2

In [ ]:
no = 4
m = 2
n = 3
Bt = np.random.rand(no, no, m)
Bt

In [ ]:
z = 10 * np.arange(no).reshape(no, 1)

(z.T @ Bt.T).squeeze().T

In [ ]:
np.vstack([z.T @ Bt[:, :, 0].T, z.T @ Bt[:, :, 1].T])

In [ ]:
zt = np.hstack([z + i for i in range(n)])
print(zt)
zt.T @ Bt.T

In [ ]:
lt = np.arange(n * no).reshape(no, n)
lt

In [ ]:
for i in range(n):
    print(
        np.vstack([zt[:, i].T @ Bt[:, :, 0].T, zt[:, i].T @ Bt[:, :, 1].T]) @ lt[:, i]
    )

In [ ]:
np.einsum("ijk,kj->ij", zt.T @ Bt.T, lt)

### Misc

In [ ]:
np.repeat(np.array([[-1, 1], [-2, 2]]).T, 3, axis=1).T

In [ ]:
L = np.random.rand(4, 4)
y = np.random.rand(4, 1)
ut = np.random.rand(2)

Bt.T @ ut @ y

In [ ]:
np.einsum("ijk,i->jk", Bt.T, ut)

In [ ]:
Bt[:, :, 0].T * ut[0] + Bt[:, :, 1].T * ut[1]